In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary
import numpy as np
import os

In [2]:
# create a fully connected network

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        
        self.Linear = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Linear(50, num_classes)
        )
        
    def forward(self, x):
        x = self.Linear(x)
        return x

In [3]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape)

torch.Size([64, 10])


In [4]:
device = "cpu"
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [5]:
train_dataset = datasets.MNIST('D:/Datasets/Torch_MNIST/', 
                               train = True, 
                               transform = transforms.ToTensor(),
                               download = True)

train_loader = DataLoader(dataset = train_dataset, 
                          batch_size = batch_size, 
                          shuffle = True)

In [6]:
test_dataset = datasets.MNIST('D:/Datasets/Torch_MNIST/', 
                               train = False, 
                               transform = transforms.ToTensor(),
                               download = True)

test_loader = DataLoader(dataset = test_dataset, 
                          batch_size = batch_size, 
                          shuffle = True)

In [7]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [15]:
for epoch in range(num_epochs):
    for batch_ix, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        data = data.reshape(data.shape[0], -1)
        
        scores = model(data)
        loss = criterion(scores, target)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
print("Training Completed")

Training Completed


In [18]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print("accuracy : ", (float(num_correct) / float(num_samples)) * 100)

In [19]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

accuracy :  94.80333333333333
accuracy :  94.41000000000001
